In [3]:
import numpy as np
import os
import re
from byte_tracker import BYTETracker
# from nets import nn

In [4]:
print('hello')
file_path = 'detected_objects_with_fps.txt'

hello


In [ ]:
scores = []
coordinates = []
labels = []

def read_txt():
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            print(parts)
            if len(parts) == 6:
                label = int(parts[0])
                confidences = float(parts[-1])
                coord = [float(x) for x in parts[1:5]]
                
                labels.append(label)
                scores.append(confidences)
                coordinates.append(coord)

# def read_txt_old():
    # with open(file_path, 'r') as file:
    #     text = file.read()
    #     pattern = r'(\w+): \[(\d+\.\d+)\] \(([\d,\s]+)\)'
    #     matches = re.findall(pattern, text)

    #     # Parse and append values to respective lists
    # for match in matches:
    #     label, confidences, coord = match
    #     scores.append(float(confidences))
    #     coord = [float(x) for x in coord.split(',')]
    #     coordinates.append(coord)
    #     labels.append(label)

    #     # for line in file:
    #     #     line = line.rstrip(os.linesep)

    #     #     if (len(line) == 0):
    #     #         continue
    #     #     else:
                
    # return 'asdf'

read_txt()

In [7]:
detections = []
i = 0
for coor in coordinates:
    l, t = float(coor[0]), float(coor[1])
    r, b = float(coor[2]), float(coor[3])

    conf = scores[i]
    i += 1

    detections.append([l, t, r, b, conf])

detections = np.array(detections, dtype=float)

In [10]:

print(detections)

[[0.960773  0.470833  0.0175644 0.0875    0.95     ]
 [0.684426  0.739583  0.0503513 0.1125    0.95     ]
 [0.0456674 0.386458  0.0351288 0.04375   0.95     ]
 [0.441452  0.346875  0.0983607 0.0895833 0.95     ]
 [0.242974  0.358333  0.0854801 0.075     0.95     ]
 [0.339578  0.705208  0.124122  0.185417  0.95     ]
 [0.357143  0.367708  0.154567  0.11875   0.95     ]
 [0.609485  0.342708  0.0925059 0.102083  0.95     ]
 [0.683841  0.738542  0.0515222 0.114583  0.95     ]
 [0.0468384 0.385417  0.0351288 0.0458333 0.95     ]
 [0.444379  0.346875  0.0925059 0.09375   0.95     ]
 [0.246487  0.359375  0.0831382 0.0729167 0.95     ]
 [0.342506  0.705208  0.129977  0.185417  0.95     ]
 [0.359485  0.366667  0.156909  0.116667  0.95     ]
 [0.617682  0.34375   0.113583  0.104167  0.95     ]
 [0.0480094 0.385417  0.0351288 0.0458333 0.95     ]
 [0.683841  0.736458  0.0515222 0.11875   0.95     ]
 [0.447307  0.347917  0.0866511 0.0916667 0.95     ]
 [0.248829  0.358333  0.0854801 0.075     0.95

In [9]:
np.float = float


class BYTETrackerArgs:
    def __init__(self):
        self.track_thresh = 0.25
        self.track_buffer = 30
        self.match_thresh = 0.8
        self.aspect_ratio_thresh = 3.0
        self.min_box_area = 1.0
        self.mot20 = False

byte_tracker_args = BYTETrackerArgs()
byte_tracker = BYTETracker(byte_tracker_args)

tracks_result = byte_tracker.update(
        output_results=np.array(detections, dtype=float),
        img_info=(640, 240),
        img_size=(640, 240)
    )


In [17]:
# for track_item in tracks_result:
#     print(track_item.track_id)
#     print(track_item._tlwh)

print(tracks_result[0].tlbr)
print(tracks_result[0].tlwh)
print(tracks_result[0].track_id)
print(tracks_result[0].score)
print(tracks_result[0].location)

[0.423958  0.843677  0.5188058 0.9207603]
[0.423958  0.843677  0.0948478 0.0770833]
110
0.95
(inf, inf)


In [ ]:
def iou(box: np.ndarray, boxes: np.ndarray):
    xy_max = np.minimum(boxes[:, 2:], box[2:])
    xy_min = np.maximum(boxes[:, :2], box[:2])
    inter = np.clip(xy_max-xy_min, a_min=0, a_max=np.inf)
    inter = inter[:, 0]*inter[:, 1]
    
    area_boxes = (boxes[:, 2]-boxes[:, 0])*(boxes[:, 3]-boxes[:, 1])
    area_box = (box[2]-box[0])*(box[3]-box)
    

In [18]:
import os
from byte_tracker import STrack

dir_path = r"./exp/label"
files = os.listdir(dir_path)

for file_path in files:
    print("new file")
    labels = []
    scores = []
    coordinates = []
    
    # --- read file ---
    with open(os.path.join(dir_path, file_path), 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 6:
                label = int(parts[0])
                confidences = float(parts[-1])
                coord = [float(x) for x in parts[1:5]]
                
                labels.append(label)
                scores.append(confidences)
                coordinates.append(coord)
            if len(parts) == 5:
                label = int(parts[0])
                confidences = 0.95
                coord = [float(x) for x in parts[1:5]]
                
                labels.append(label)
                scores.append(confidences)
                coordinates.append(coord)
              
    # --- detect ---
    detections = []
    i = 0
    for coor in coordinates:
        tlwh = np.array([float(coor[1]), float(coor[0]), float(coor[2]), float(coor[3])])
        detect = STrack.tlwh_to_tlbr(tlwh)
        conf = scores[i]
        i += 1

        detect = np.append(detect, conf)
        detections.append(detect)

    detections = np.array(detections, dtype=float)
    
    # --- ByteTracker ---
    np.float = float

    class BYTETrackerArgs:
        def __init__(self):
            self.track_thresh = 0.25
            self.track_buffer = 30
            self.match_thresh = 0.8
            self.aspect_ratio_thresh = 3.0
            self.min_box_area = 1.0
            self.mot20 = False

    byte_tracker_args = BYTETrackerArgs()
    byte_tracker = BYTETracker(byte_tracker_args)

    tracks_result = byte_tracker.update(
            output_results=np.array(detections, dtype=float),
            img_info=(640, 240),
            img_size=(640, 240)
        )
    
    for track_item in tracks_result:
        print(track_item.track_id)

new file
118
119
120
121
122
123
124
125
new file
126
127
128
129
130
131
132
new file
133
134
135
136
137
138
139
new file
140
141
142
143
144
145
146
147
148
new file
149
150
151
152
153
154
155
156
